In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip
/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip
/kaggle/input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv
/kaggle/input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels.h5
/kaggle/input/resnet50/imagenet_class_index.json
/kaggle/input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5


In [18]:
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

https://www.kaggle.com/keras/resnet50?select=resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5

In [19]:
path = '../input/dogs-vs-cats-redux-kernels-edition/'
weight_loc = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [20]:
train_img_path = os.path.join(path,'train.zip')
test_img_path = os.path.join(path,'test.zip')

In [21]:
from zipfile import ZipFile
with ZipFile(train_img_path,'r') as zip:
    zip.extractall('.')

with ZipFile(test_img_path,'r') as zip:
    zip.extractall('.')

In [22]:
print(os.listdir('../'))
print(os.listdir('../working'))

['lib', 'input', 'working']
['test', 'train', '__notebook_source__.ipynb']


In [23]:
filename = os.listdir('../working/train')
labels = []
for file in filename:
    category = file.split('.')[0]
    if category == 'cat':
        labels.append(0)
    else:
        labels.append(1)
        
df = pd.DataFrame({'filename': filename, 'label': labels})
df.head()

,filename,label
0,cat.9799.jpg,0
1,dog.1506.jpg,1
2,dog.8875.jpg,1
3,dog.3551.jpg,1
4,dog.2239.jpg,1


In [ ]:
def show_img(data, isTest = False):
    fig, ax = plt.subplots(nrows = 5, ncols = 5, figsize = (5,5))
    for i,data in 